## Tensorflow 学习

In [2]:
import tensorflow as tf 

- 正则化处理

In [5]:
w1 = tf.random.normal((4, 3))
w2 = tf.random.normal((4, 2))

loss_reg = tf.reduce_sum(tf.abs(w1)) + tf.reduce_sum(tf.abs(w2))
print(loss_reg) # 正则化处理

tf.Tensor(14.03604, shape=(), dtype=float32)


- 卷积运算
  
  感受野与卷积核**逐位相乘**后累加

In [15]:
# 输入格式为：batch_shape + [in_height, in_width, in_channels]
x = tf.random.normal([2, 5, 5, 3]) # 模拟输入，3通道，高宽为5
# 创建4个3x3的卷积核，格式为[filter_height, filter_width, in_channels, out_channels]
w = tf.random.normal([3, 3, 3, 4]) 

## 1. padding为0，步长为1
out1 = tf.nn.conv2d(input=x, filters=w, strides=1, padding=[[0, 0], [0, 0], [0, 0], [0, 0]])
print("out1: ", out1.shape)

## 2. 上下左右各填充1个单位，步长为1
#    when data_format is `"NHWC"`, padding should be in the form 
#           `[[0, 0], [pad_top, pad_bottom], [pad_left, pad_right], [0, 0]]`. 
#    When explicit padding used and data_format is `"NCHW"`, this should be in the form 
#           `[[0, 0], [0, 0], [pad_top, pad_bottom], [pad_left, pad_right]]`.
out2 = tf.nn.conv2d(input=x, filters=w, strides=1, padding=[[0, 0], [1, 1], [1, 1], [0, 0]])
print("out2: ", out2.shape)

## 3. padding="SAME", 输出维度的高宽成1/s倍减少
out3 = tf.nn.conv2d(input=x, filters=w, strides=2, padding="SAME")
print("out3: ", out3.shape)

out1:  (2, 3, 3, 4)
out2:  (2, 5, 5, 4)
out3:  (2, 3, 3, 4)


- 卷积层类

`Conv2D`: 卷积层类，里面保存了张量W和偏置b，可以通过`trainable_variables`、`kernel`、`bias`等成员获取

In [ ]:
## 1. 创建4个3x3大小的卷积核层
layer1 = tf.keras.layers.Conv2D(4, kernel_size=3, strides=1, padding="SAME")

## 2. 创建4个高x宽为3x4大小的卷积核，高宽方向的步长为2和1
layer2 = tf.keras.layers.Conv2D(4, kernel_size=(3, 4), strides=(2, 1), padding="SAME")

out = layer1(x) # 前向计算
print(out.shape)

- 池化层

从局部相关的一组元素中采样或进行信息聚合，从而得到新的元素值

最大池化（Max Pooling）：最大值
平均池化（Average Pooling）：平均值

- BatchNormal层
  
  测试模式与训练模式要区分

In [ ]:
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers, optimizers

# 2 images with 4x4 size, 3 channels
x = tf.random.normal([2, 4, 4, 3], mean=.1, stddev=0.5)

net = layers.BatchNormalization(axis=-1, center=True, scale=True, trainable=True)

out = net(x)
print("forward in test model: ", net.variables)

out = net(x, training=True)
print("forward in train model(1 step): ", net.variables)


In [ ]:
for i in range(100):
    out = net(x, training=True)
print('forward in train mode(100 steps):', net.variables)


optimizer = optimizers.SGD(lr=1e-2)
for i in range(10):
    with tf.GradientTape() as tape:
        out = net(x, training=True)
        loss = tf.reduce_mean(tf.pow(out, 2)) - 1

    grads = tape.gradient(loss, net.trainable_variables)
    optimizer.apply_gradients(zip(grads, net.trainable_variables))
print('backward(10 steps):', net.variables)


- CIFAR10训练

In [3]:
from tensorflow.keras import layers, optimizers, datasets, Sequential
import os 

(x, y), (x_test, y_test) = datasets.cifar10.load_data()
y = tf.squeeze(y, axis=1)
y_test = tf.squeeze(y_test, axis=1)
print(x.shape, y.shape, x_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000,) (10000, 32, 32, 3) (10000,)


2022-07-22 11:13:26.091663: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/yeting/.mujoco/mujoco210/bin
2022-07-22 11:13:26.091740: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-22 11:13:26.091819: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: eb23ed7a4bcf
2022-07-22 11:13:26.091847: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: eb23ed7a4bcf
2022-07-22 11:13:26.091999: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: NOT_FOUND: was unable to find libcuda.so DSO loaded into this program
2022-07-22 11:13:26.092106: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 450.80.2
2022-07-22 11:13:26.098730: I te

In [4]:
def preprocess(x, y):
    # [0, 1]
    x = 2*tf.cast(x, dtype=tf.float32) / 255. - 1
    y = tf.cast(y, dtype=tf.int32)
    return x, y

train_db = tf.data.Dataset.from_tensor_slices((x, y))
train_db = train_db.shuffle(1000).map(preprocess).batch(128)

test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_db = test_db.map(preprocess).batch(64)

In [6]:
conv_layers = [ # 5 units of conv + max pooling
    # unit 1
    layers.Conv2D(64, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(64, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding="same"),
    
    # unit 2
    layers.Conv2D(128, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(128, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding="same"),

    # unit 3
    layers.Conv2D(256, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(256, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding="same"),

    # unit 4
    layers.Conv2D(512, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(512, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding="same"),

    # unit 5
    layers.Conv2D(512, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(512, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding="same"),
]

def main():
    # [b, 32, 32, 3] => [b, 1, 1, 512]
    conv_net = Sequential(conv_layers)

    fc_net = Sequential([
        layers.Dense(256, activation=tf.nn.relu),
        layers.Dense(128, activation=tf.nn.relu),
        layers.Dense(10, activation=None),
    ])

    conv_net.build(input_shape=[None, 32, 32, 3])
    fc_net.build(input_shape=[None, 512])
    conv_net.summary()
    fc_net.summary()

    optimizer = optimizers.Adam(lr=1e-4)
    
    # [1, 2] + [3, 4] => [1, 2, 3, 4]
    variables = conv_net.trainable_variables + fc_net.trainable_variables

    for epoch in range(50):
        for step, (x, y) in enumerate(train_db):
            with tf.GradientTape() as tape:
                # [b, 32, 32, 3] => [b, 1, 1, 512]
                out = conv_net(x)
                # flatten, => [b, 512]
                out = tf.reshape(out, [-1, 512])
                # [b, 512] => [b, 10]
                logits = fc_net(out)
                # [b] => [b, 10]
                y_onehot = tf.one_hot(y, depth=10)
                # compute loss
                loss = tf.losses.categorical_crossentropy(y_onehot, logits, from_logits=True)
                loss = tf.reduce_mean(loss)
            grads = tape.gradient(loss, variables)
            optimizer.apply_gradients(zip(grads, variables)) 

            if step % 100 == 0:
                print(epoch, step, "loss: ", float(loss))
        
        # test
        total_num = 0
        total_correct = 0
        for x, y in test_db:
            out = conv_net(x)
            out = tf.reshape(out, [-1, 512])
            logits = fc_net(out)
            prob = tf.nn.softmax(logits, axis=1)
            pred = tf.argmax(prob, axis=1)
            pred = tf.cast(pred, dtype=tf.int32)

            correct = tf.cast(tf.equal(pred, y), dtype=tf.int32)
            correct = tf.reduce_sum(correct)

            total_num += x.shape[0]
            total_correct += int(correct)
        
        acc = total_correct / total_num
        print(epoch, " acc: ", acc) 

main()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 32, 32, 64)        1792      
                                                                 
 conv2d_11 (Conv2D)          (None, 32, 32, 64)        36928     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 16, 16, 64)       0         
 2D)                                                             
                                                                 
 conv2d_12 (Conv2D)          (None, 16, 16, 128)       73856     
                                                                 
 conv2d_13 (Conv2D)          (None, 16, 16, 128)       147584    
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 8, 8, 128)        0         
 2D)                                                  

- 分离卷积 (Depth-wise Separable Convolution)

卷积核的每个通道与输入的每个通道做卷积运算，得到多个通道的中间特征，然后这些特征与多个1x1卷积核进行普通卷积运算，这些输出在通道轴上进行拼接
优点：相同的输入输出，可以减少参数量